# 시계열로 PA예측하기(with fbProphet) - 오태환

## 0) Importing Packages and Batter

In [ ]:
import pandas as pd

In [ ]:
from fbprophet import Prophet
import datetime

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/games_merged.csv")

In [ ]:
df['G'] = df['HOME_G'] + df['VISIT_G']

In [ ]:
df['PA_PER_G'] = df['PA'] / df['G']

In [ ]:
df = df[['year', 'month', 'P_ID', 'PA_PER_G']]

In [ ]:
df = df.loc[df.month > 4 ,:]

In [ ]:
df = df.loc[df.month < 10,:]

In [ ]:
pplayer = df.loc[df.year == 2020,:]

In [ ]:
df['ds'] = df.agg(lambda x: f"{int(x['year'])}-{int(x['month'])}-30", axis=1)

In [ ]:
df

In [ ]:
len(pplayer)

In [ ]:
pplayer = pplayer['P_ID'].unique()

In [ ]:
pplayer

In [ ]:
import numpy as np

In [ ]:
predicted = pd.DataFrame()

for i in range(len(pplayer)):
    print("process : ", i/len(pplayer))
    temp = df[df['P_ID'].isin([pplayer[i]])]
    temp['y'] = temp['PA_PER_G']
    temp = temp[['ds', 'y']]
    temp['floor'] = 0
    temp.fillna(0)
    if len(temp) > 2:
        m = Prophet(seasonality_mode='multiplicative', mcmc_samples=300).fit(temp)
        future = m.make_future_dataframe(periods=3, freq='M')
        future['floor'] = 0
        fcst = m.predict(future)
        fcst = fcst[fcst.ds > '2020-07-01']
        fcst['PA_PER_G'] = fcst['yhat']
        fcst['P_ID'] = pplayer[i]
        fcst = fcst[['ds','P_ID', 'PA_PER_G']]
        fcst = pd.DataFrame(fcst)
        fcst = fcst.reset_index(drop = True)
        fcst.PA_PER_G[fcst.PA_PER_G > np.max(temp['y'])] = np.max(temp['y'])
        last = fcst.iloc[-1][-1]
        octo = pd.DataFrame(['2020-10-31', pplayer[i],last])
        octo = np.transpose(octo)
        octo.columns = ['ds', 'P_ID', 'PA_PER_G']
        fcst = fcst.append(octo)
        fcst = fcst.reset_index(drop = True)
        predicted = predicted.append(fcst, ignore_index=True)
    else:
        ds = pd.DataFrame(['2020-07-31', '2020-08-31', '2020-09-30', '2020-10-31'])
        ds['P_ID'] = pplayer[i]
        ds['PA_PER_G'] = np.mean(temp['y'])
        ds.columns = ['ds', 'P_ID', 'PA_PER_G']
        predicted = predicted.append(ds, ignore_index=True)
        

In [ ]:
import numpy as np

In [ ]:
predicted

In [ ]:
predicted.PA_PER_G[predicted.PA_PER_G < 0] = 0

In [ ]:
predicted

In [ ]:
!ls '/content/drive/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls '/content/drive/'

In [ ]:
!mkdir -p '/content/drive/My Drive/bigcon'
!ls -Fcal '/content/drive/My Drive/bigcon'

In [ ]:
!ls -Fcal '/content/drive/My Drive/bigcon'

In [ ]:
predicted.to_csv("/content/drive/My Drive/bigcon/predicted_pa_per_g.csv", index = False)

In [ ]:
predicted.to_csv("predicted_pa_per_g.csv", index = False)

In [ ]:
from google.colab import files

In [ ]:
files.download("predicted_pa_per_g.csv")

In [ ]:
np.where(pplayer == 64300)

In [ ]:
np.max(temp['y'])

In [ ]:
temp = df[df['P_ID'].isin([pplayer[199]])]
temp['y'] = temp['PA_PER_G']
temp = temp[['ds', 'y']]

temp.fillna(0)

m = Prophet(mcmc_samples=300).fit(temp)
future = m.make_future_dataframe(periods=3, freq='M')

fcst = m.predict(future)

m.plot(fcst)

In [ ]:
fcst = fcst[fcst.ds > '2020-07-01']
fcst['PA_PER_G'] = fcst['yhat']
fcst['P_ID'] = pplayer[i]
fcst = fcst[['ds','P_ID', 'PA_PER_G']]
fcst = pd.DataFrame(fcst)
fcst = fcst.reset_index(drop = True)
fcst.PA_PER_G[fcst.PA_PER_G > np.max(temp['y'])] = np.max(temp['y'])

In [ ]:
last = fcst.iloc[3][2]

In [ ]:
octo = pd.DataFrame(['2020-10-31', pplayer[199],last])
octo = np.transpose(octo)
octo.columns = ['ds', 'P_ID', 'PA_PER_G']

In [ ]:
fcst.append(octo)